In [1]:
import numpy as np
import keras.backend as K
import matplotlib.pyplot as plt
import re
import tarfile

In [2]:
from keras.models import Model
from keras.layers import Dense, Embedding, Input, Lambda, Reshape, add, dot, Activation
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import rmsprop_v2
from keras.utils.data_utils import get_file

In [3]:
path = get_file(
    'babi-tasks-v1-2.tar.gz',
    origin='https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz')
tar = tarfile.open(path)

11755520/11745123 [==============================] - 52s 4us/step


In [18]:
challenges = {
  # QA1 with 10,000 samples
    'single_supporting_fact_10k': 'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt',
    # QA2 with 10,000 samples
    'two_supporting_facts_10k': 'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt',   
}

In [19]:
def tokenize(sent):
    '''
    Return the tokens of a sentence including punctuation
     >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the',          'apple', '?']
  
    '''

    return [x.strip() for x in re.split('(\W+)', sent) if x.strip()]

In [36]:
def get_stories(f):
    # data will return a list of triples
    # each triple contains:
    #   1. a story
    #   2. a question about the story
    #   3. the answer to the question
    data = []

    # use this list to keep track of the story so far
    story = []

    # print a random story, helpful to see the data
    printed = False
    for line in f:
        line = line.decode('utf-8').strip()

        # split the line number from the rest of the line
        nid, line = line.split(' ', 1)

        # see if we should begin a new story
        if int(nid) == 1:
            story = []

        # this line contains a question and answer if it has a tab
        #       question<TAB>answer
        # it also tells us which line in the story is relevant to the answer
        # Note: we actually ignore this fact, since the model will learn
        #       which lines are important
        # Note: the max line number is not the number of lines of the story
        #       since lines with questions do not contain any story
        # one story may contain MULTIPLE questions
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)

            # numbering each line is very useful
            # it's the equivalent of adding a unique token to the front
            # of each sentence
            story_so_far = [[str(i)] + s for i, s in enumerate(story) if s]

            # uncomment if you want to see what a story looks like
            # if not printed and np.random.rand() < 0.5:
            #     print("story_so_far:", story_so_far)
            #     printed = True
            data.append((story_so_far, q, a))
            story.append('')
        else:
            # just add the line to the current story
            story.append(tokenize(line))
    return data

In [28]:
# recursively flatten a list
def should_flatten(el):
    return not isinstance(el, (str, bytes))

In [29]:
def flatten(l):
    for el in l:
        # if el is not a string, yield it from flatten
        if should_flatten(el):
            yield from flatten(el)
        else:
            yield el


In [30]:
# convert stories from words into lists of word indexes(integers)
# pad each sequence so that they are the same length
# we will need to re-pad the stories later so that
# each story is the same length

def vectorize_stories(data, word2idx, story_maxlen, query_maxlen):
    inputs, queries, answers = [], [], []
    for story, query, answer in data:
        inputs.append([[word2idx[w] for w in s] for s in story])
        queries.append([word2idx[w] for w in query])
        answers.append([word2idx[answer]])

    return (
        [pad_sequences(x, maxlen=story_maxlen) for x in inputs],
        pad_sequences(queries, maxlen=query_maxlen),
        np.array(answers)
    )

In [31]:
# this is like 'pad_sequences' but for entire stories
# we are padding each story with zeros so every story
# has the same number of sentences
# append an array of zeros of size:
# (max_sentences - num sentences in story, max words in sentence)
def stack_inputs(inputs, story_maxsents, story_maxlen):
    for i, story in enumerate(inputs):
        inputs[i] = np.concatenate(
            [
                story,
                np.zeros(
                    (story_maxsents - story.shape[0], story_maxlen),
                    'int'
                )
            ]
        )
    return np.stack(inputs)

In [38]:
# make a function to get the data since
# we want to load both the single supporting fact data
# and the two supporting fact data later

def get_data(challenge_type):
    # input should either be 'single_supporting_fact_10k' or 'two_supporting_facts_10k'
    challenge = challenges[challenge_type]

    # return a list of triplets
    # (story, ques, ans)

    train_stories = get_stories(tar.extractfile(challenge.format('train')))
    test_stories = get_stories(tar.extractfile(challenge.format('test')))

    # group all stories together
    stories = train_stories + test_stories

    # so we can get the max length of each story, of each sentence, and of each question
    story_maxlen = max((len(s) for x, _, _ in stories for s in x))
    story_maxsents = max((len(x) for x, _, _ in stories))
    query_maxlen = max(len(x) for _, x, _ in stories)

    # Create vocabulary of corpus and find size, including a padding element
    vocab = sorted(set(flatten(stories)))
    vocab.insert(0, '<PAD>')
    vocab_size = len(vocab)

    # Create index mapping for the vocab
    word2idx = {c: i for i, c in enumerate(vocab)}

    # convert stories from strings to lists of integers
    input_train, queries_train, answers_train = vectorize_stories(
        train_stories,
        word2idx,
        story_maxlen,
        query_maxlen
    )
    input_test, queries_test, answers_test = vectorize_stories(
        test_stories,
        word2idx,
        story_maxlen,
        query_maxlen
    )

    # convert inputs into 3-D numpy arrays
    inputs_train = stack_inputs(input_train, story_maxsents, story_maxlen)
    inputs_test = stack_inputs(input_test, story_maxsents, story_maxlen)
    print(f'inputs_train.shape : {inputs_train.shape}, inputs_test.shape: {inputs_test.shape}' )

    # return model inputs for keras

    return train_stories, test_stories, \
        inputs_train, queries_train, answers_train, \
        inputs_test, queries_test, answers_test, \
        story_maxsents, story_maxlen, query_maxlen, \
        vocab, vocab_size

In [39]:
# get the single supporting fact data
train_stories, test_stories, \
    inputs_train, queries_train, answers_train, \
    inputs_test, queries_test, answers_test, \
    story_maxsents, story_maxlen, query_maxlen, \
    vocab, vocab_size = get_data('single_supporting_fact_10k')

inputs_train.shape : (10000, 10, 8), inputs_test.shape: (1000, 10, 8)
